In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
%pip install langchain_openai

  Using cached langchain_openai-0.3.28-py3-none-any.whl.metadata (2.3 kB)
Using cached langchain_openai-0.3.28-py3-none-any.whl (70 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [19]:
db_cols=['protocol_id','node_type','q_id','questions','options','diagnosis','red_flag_symptom','over_to_counter','advice','precaution','lab_tests','diet','description','age_category','gender']



In [20]:
import json

fp=open('cough_latest_complete.json','r')
qna_data=json.load(fp)
qna_data

# Examine age_group AND gender distribution
for dic in qna_data['questions']:
    if 'age_group' not in dic:
        print(f"No age_group: {dic.get('q_id', 'Unknown')} - Gender: {dic.get('gender', 'Unknown')}")
    else:
        print(f"Age: {dic['age_group']} - Gender: {dic.get('gender', 'Unknown')} - Q_ID: {dic.get('q_id', 'Unknown')}")

Age: None - Gender: Both - Q_ID: 1
Age: None - Gender: Both - Q_ID: 2
Age: 0-2 - Gender: Both - Q_ID: 3
Age: 0-2 - Gender: Both - Q_ID: 4
Age: 0-2 - Gender: Both - Q_ID: 5
Age: 0-2 - Gender: Both - Q_ID: 6
Age: 0-2 - Gender: Both - Q_ID: 7
Age: 0-2 - Gender: Both - Q_ID: 8
Age: 0-2 - Gender: Both - Q_ID: 9
Age: 0-2 - Gender: Both - Q_ID: 10
Age: 0-2 - Gender: Both - Q_ID: 11
Age: 0-2 - Gender: Both - Q_ID: 12
Age: 0-2 - Gender: Both - Q_ID: 13
Age: 0-2 - Gender: Both - Q_ID: 14
Age: 3-12 - Gender: Both - Q_ID: 15
Age: 3-12 - Gender: Both - Q_ID: 16
Age: 3-12 - Gender: Both - Q_ID: 17
Age: 3-12 - Gender: Both - Q_ID: 18
Age: 3-12 - Gender: Both - Q_ID: 19
Age: 3-12 - Gender: Both - Q_ID: 20
Age: 3-12 - Gender: Both - Q_ID: 21
Age: 3-12 - Gender: Both - Q_ID: 22
Age: 3-12 - Gender: Both - Q_ID: 23
Age: 3-12 - Gender: Both - Q_ID: 24
Age: 3-12 - Gender: Both - Q_ID: 25
Age: 3-12 - Gender: Both - Q_ID: 26
Age: 13-18 - Gender: Both - Q_ID: 27
Age: 13-18 - Gender: Both - Q_ID: 28
Age: 13-18 

In [21]:
import json

fp=open('cough_latest_complete.json','r')
qna_data=json.load(fp)
qna_data

{'questions': [{'q_id': 1,
   'q_tag': 'DEMOGRAPHICS',
   'question': 'What is your gender?',
   'options': [{'option_id': 1, 'opt_value': 'Male'},
    {'option_id': 2, 'opt_value': 'Female'}],
   'age_group': None,
   'gender': 'Both',
   'clinical_rationale': 'Essential demographic information for accurate medical assessment'},
  {'q_id': 2,
   'q_tag': 'DEMOGRAPHICS',
   'question': 'What is your age category?',
   'options': [{'option_id': 1, 'opt_value': '0-2 years (Infant/Toddler)'},
    {'option_id': 2, 'opt_value': '3-12 years (Child)'},
    {'option_id': 3, 'opt_value': '13-18 years (Adolescent)'},
    {'option_id': 4, 'opt_value': '19-40 years (Young Adult)'},
    {'option_id': 5, 'opt_value': '41-65 years (Middle-aged Adult)'},
    {'option_id': 6, 'opt_value': '66+ years (Senior)'}],
   'age_group': None,
   'gender': 'Both',
   'clinical_rationale': 'Age-specific medical assessment and treatment considerations'},
  {'q_id': 3,
   'q_tag': 'SYMPTOMS',
   'question': 'Does t

In [22]:
# Create age + gender wise dictionary
age_gender_wise_dict = {}

for dic in qna_data['questions']:
    age_group = dic['age_group']
    gender = dic.get('gender', 'Both')
    
    if age_group:  # Skip None age_group (demographic questions)
        # Create key as "age_group-gender"
        key = f"{age_group}-{gender}"
        
        if key not in age_gender_wise_dict:
            age_gender_wise_dict[key] = []
        age_gender_wise_dict[key].append(dic['question'])

# Print the results
for key, questions in age_gender_wise_dict.items():
    print(f"\n=== {key} ===")
    print(f"Total questions: {len(questions)}")
    for i, question in enumerate(questions, 1):
        print(f"{i}. {question}")


=== 0-2-Both ===
Total questions: 12
1. Does the infant have a persistent cough?
2. Is the cough accompanied by fever?
3. Does the infant have difficulty breathing?
4. Is the cough wet or productive?
5. Does the infant have nasal congestion?
6. Is the infant's cough worse at night?
7. Is the infant wheezing while coughing?
8. Has the infant been exposed to smoke or pollutants?
9. Does the infant have a history of allergies?
10. Does the infant vomit after coughing?
11. Is the infant feeding normally despite the cough?
12. Has the infant been recently vaccinated?

=== 3-12-Both ===
Total questions: 12
1. Does the child have a persistent cough?
2. Is the cough accompanied by a sore throat?
3. Does the child have a fever along with the cough?
4. Is the cough dry or productive?
5. Does the child experience chest pain while coughing?
6. Does the child have a runny nose?
7. Is the cough worse in the morning or at night?
8. Does the child have a history of asthma or allergies?
9. Does the ch

In [ ]:
# # schemas 

# from pydantic import BaseModel, Field
# from typing import List, Optional

# class option(BaseModel):
#     opt_id: int = Field(description="Unique identifier for the option")
#     opt_value: str = Field(description="The actual option value or text")
#     next_q_id: int = Field(description="Foreign key reference to the question ID (q_id)")

# class DT_schema(BaseModel):
#     id: Optional[int] = Field(None, description="Unique identifier for the DT schema")

# class Reasoning(BaseModel):
#     q_id: int = Field(description="Question ID being reasoned about")
#     opt_id: int = Field(description="Option ID selected for reasoning")
#     next_q_id: int = Field(description="Next question ID in the reasoning flow")
#     rule: str = Field(description="The clinical rule for mapping next question to ask the patient. next question can be question or diagnosis from previous questions")
#     validation: str = Field(description="Validate the next question is clinicaly correct and required., if it is diagnosis, map the clinically correct diagnosis")
#     conclusion: str = Field(description="Final conclusion from the reasoning, implement correct next question ID (q_id).")

# class DB_schema(BaseModel):
#     gender: str = Field(description="Gender category of the participant")
#     age_group: str = Field(description="Age group classification")
#     questionnaire: List[questionnaire] = Field(description="List of associated questionnaires")
#     q_id: int = Field(description="Question identifier")
#     question: str = Field(description="The actual question text")
#     options: List[option] = Field(description="List of available options for the question")
#     q_tag: str = Field(description="Question tag or category")

# class DB_records(BaseModel):
#     DB_records: List[DT_schema] = Field(description="List of database record schemas")
#     reasoning_steps: List[Reasoning] = Field(description="List of reasoning steps for each question ID(q_id).")

In [ ]:
# from pydantic import BaseModel, Field
# from typing import List, Optional
# from enum import Enum

# class option(BaseModel):
#     opt_id: int = Field(description="Unique identifier for this option within the question")
#     opt_value: str = Field(description="The option text displayed to the patient (e.g., 'Yes', 'No', 'Severe pain')")
#     next_q_id: int = Field(description="The next question ID (q_id) that this option leads to - can point to either a question node or diagnosis node")

# class Reasoning(BaseModel):
#     q_id: int = Field(description="Current question ID being processed in the decision tree")
#     opt_id: int = Field(description="Option ID from the current question that triggers this reasoning path")
#     next_q_id: int = Field(description="The clinically determined next question ID that this option should lead to")
#     rule: str = Field(description="Clinical decision rule explaining why this option leads to the next question/diagnosis (e.g., 'If patient reports severe chest pain, immediately assess for cardiac symptoms')")
#     validation: str = Field(description="Clinical validation confirming the next step is medically appropriate and follows established protocols. For diagnosis nodes, validate the diagnosis is clinically accurate based on symptom pattern")
#     conclusion: str = Field(description="Final reasoning outcome confirming the correct next_q_id mapping with clinical justification")

# class DB_schema(BaseModel):
   
#     node_type: str = Field(description="Type of node in decision tree - 'question' for symptom inquiry or 'diagnosis' for final medical conclusion")
#     q_id: int = Field(description="Unique question/node identifier within the decision tree sequence")
#     questions: str = Field(description="The medical question text presented to patient (e.g., 'Do you have chest pain?')")
#     options: List[option] = Field(description="Available response options for this question, each mapped to next clinical step")
#     diagnosis: Optional[str] = Field(None, description="Medical diagnosis conclusion (only populated for diagnosis nodes)")
#     red_flag_symptom: Optional[str] = Field(None, description="Critical symptoms requiring immediate medical attention")
#     over_to_counter: Optional[str] = Field(None, description="Recommended over-the-counter medications if applicable")
#     advice: Optional[str] = Field(None, description="General medical advice or guidance for the patient")
#     precaution: Optional[str] = Field(None, description="Important precautions or warnings the patient should follow")
#     lab_tests: Optional[str] = Field(None, description="Recommended laboratory tests or diagnostic procedures")
#     diet: Optional[str] = Field(None, description="Dietary recommendations or restrictions")
#     description: Optional[str] = Field(None, description="Additional descriptive information about the condition or step")
#     age_category: str = Field(description="Target age group for this decision path (e.g., '0-2','3-12','13-18','19-40','41-65','66+')")
#     gender: str = Field(description="Target gender for this decision path (e.g., 'Male', 'Female', 'Both')")

# class DecisionTreeRecords(BaseModel):
#     """Complete decision tree record set for database insertion"""
#     db_records: List[DB_schema] = Field(description="Complete list of decision tree nodes (questions and diagnoses) for database insertion")
#     reasoning_steps: List[Reasoning] = Field(description="Clinical reasoning validation for each option-to-next-question mapping in the decision tree")

In [24]:
from pydantic import BaseModel, Field
from typing import List, Optional

class option(BaseModel):
    next_q_id: int = Field(description="The next question ID that this option leads to")
    opt_value: str = Field(description="The option text displayed to the patient")
    option_id: int = Field(description="Unique identifier for this option within the question")

class diagnosis_info(BaseModel):
    description: str = Field(description="Detailed description of the medical condition")
    diagnosis_title: str = Field(description="Name/title of the diagnosis")

class DecisionTreeNode(BaseModel):
    q_id: int = Field(description="Unique question/node identifier")
    q_tag: str = Field(description="Node type: 'NEW' for questions or 'DIAGNOSIS' for diagnosis nodes")
    question: str = Field(description="The question text or diagnosis statement")
    options: Optional[List[option]] = Field(default=[], description="Available response options (empty for diagnosis nodes)")
    advice: Optional[List[str]] = Field(default=[], description="Medical advice list")
    diagnosis: Optional[diagnosis_info] = Field(default=None, description="Diagnosis information (only for diagnosis nodes)")
    lab_tests: Optional[List[str]] = Field(default=[], description="Recommended laboratory tests")
    red_flags: Optional[List[str]] = Field(default=[], description="Warning symptoms requiring immediate attention")
    precautions: Optional[List[str]] = Field(default=[], description="Precautionary measures")
    otc_medication: Optional[List[str]] = Field(default=[], description="Over-the-counter medications")

class Reasoning(BaseModel):
    q_id: int = Field(description="Current question ID being processed")
    option_id: int = Field(description="Option ID from the current question")
    next_q_id: int = Field(description="Next question ID this option leads to")
    rule: str = Field(description="Clinical decision rule for this pathway")
    validation: str = Field(description="Clinical validation of the decision")
    conclusion: str = Field(description="Final reasoning outcome with justification")

class DecisionTreeWithReasoning(BaseModel):
    decision_tree: List[DecisionTreeNode] = Field(description="Complete decision tree as flat array")
    reasoning_steps: List[Reasoning] = Field(description="Clinical reasoning validation (for internal use)")

class DecisionTreeOutput(BaseModel):
    decision_tree: List[DecisionTreeNode] = Field(description="Complete decision tree as flat array - final output")

In [25]:
dt_prompt="""
#Role and Objective
You are a Clinical Decision Support expert specializing in diagnostic pathway design. You excel at analyzing clinical questionnaires and creating comprehensive decision trees with validated symptom progressions across patient demographics.

#Task:
Analyze the Q&A JSON to construct a complete medical decision tree for all age groups ('0-2','3-12','13-18','19-40','41-65','66+') and genders (male, female, both).

Requirements:
- Map each option (opt_id) to clinically appropriate next question (q_id) 
- Next steps: symptom assessment questions OR diagnostic conclusions
- Zero unmapped options, no orphaned nodes
- Maintain clinical coherence and evidence-based pathways
- **IMPORTANT** : stick to the q_id from the Q&A JSON. 

#Inputs:
<Q&A JSON> 
    {qa_json}
</Q&A JSON>

#Reasoning Steps:
<Reasoning Method>
For each question node, apply this cycle:
**Rule**: Determine the most appropriate next diagnostic step based on current option's clinical significance and standard protocols.
**Validation**: Verify the next q_id follows clinical guidelines, maintains diagnostic accuracy, and preserves workflow integrity.
**Conclusion**: Finalize next question ID with clinical justification, ensuring decision tree completeness.
</Reasoning Method>

#Sample Output
#Sample Output
<Sample_output>
[
  {{
    "q_id": 1,
    "q_tag": "NEW",
    "question": "What is your gender?",
    "options": [
      {{
        "next_q_id": 2,
        "opt_value": "Male",
        "option_id": 1
      }},
      {{
        "next_q_id": 2,
        "opt_value": "Female",
        "option_id": 2
      }}
    ]
  }},
  {{
    "q_id": 1000,
    "q_tag": "DIAGNOSIS",
    "question": "Diagnosis: Common Cold",
    "options": [],
    "advice": ["Rest", "Stay hydrated"],
    "diagnosis": {{
      "description": "A viral infection causing mild cough and nasal congestion.",
      "diagnosis_title": "Common Cold"
    }},
    "lab_tests": ["None required"],
    "red_flags": ["High fever"],
    "precautions": ["Hand hygiene"],
    "otc_medication": ["Saline drops"]
  }}
]
</Sample_output>
Your Answer:
"""

# Inputs
fp=open('cough_latest_complete.json','r')
qna_data=json.load(fp)


decision_tree_prompt = PromptTemplate(template=dt_prompt,input_variables=["qa_json"])
model_name="gpt-4o"
llm = ChatOpenAI(temperature=0, model=model_name)
decision_tree_chain = decision_tree_prompt | llm.with_structured_output(DecisionTreeWithReasoning)

In [26]:
fp=open('cough_latest_complete.json','r')
qna_data=json.load(fp)

# Define age groups and genders for batching
age_groups = ['0-2', '3-12', '13-18', '19-40', '41-65', '66+']
genders = ['Male', 'Female']

all_batches = []

def filter_data_for_batch(qa_data, target_age, target_gender):
    """Filter questions and diagnoses for specific age group and gender"""
    filtered_questions = []
    filtered_diagnoses = []
    
    print(f"  Filtering for Age: {target_age}, Gender: {target_gender}")
    
    # Filter questions
    for q in qa_data['questions']:
        q_age = q.get('age_group')
        q_gender = q.get('gender')
        
        # Include if:
        # 1. Demographic questions (no age/gender restriction)
        # 2. Matches target age AND (gender is 'Both' OR matches target gender)
        if (q.get('q_tag') == 'DEMOGRAPHICS' or 
            (q_age == target_age and (q_gender == 'Both' or q_gender == target_gender))):
            filtered_questions.append(q)
            print(f"    ✓ Included Q{q.get('q_id')}: {q.get('question', '')[:50]}...")
    
    # Filter diagnoses
    for d in qa_data['diagnosis']:
        d_age = d.get('age_group')
        d_gender = d.get('gender')
        
        # Include if matches target age AND (gender is 'Both' OR matches target gender)
        if (d_age == target_age and (d_gender == 'Both' or d_gender == target_gender)):
            filtered_diagnoses.append(d)
            print(f"    ✓ Included Diagnosis: {d.get('diagnosis', {}).get('title', 'Unknown')}")
    
    print(f"  Total: {len(filtered_questions)} questions, {len(filtered_diagnoses)} diagnoses\n")
    
    return {
        "questions": filtered_questions,
        "diagnosis": filtered_diagnoses,
        "target_age": target_age,
        "target_gender": target_gender
    }

# Create batches for each age-gender combination
for age_group in age_groups:
    for gender in genders:
        print(f"🔄 Processing: {gender} {age_group}")
        
        # Filter data for this age-gender combination
        filtered_data = filter_data_for_batch(qna_data, age_group, gender)
        all_batches.append(filtered_data)
        
        print(f"📊 Batch created for {gender} {age_group}")
        print(f"   - Questions: {len(filtered_data['questions'])}")
        print(f"   - Diagnoses: {len(filtered_data['diagnosis'])}")
        print("="*50)

print(f"\n🎯 Total batches created: {len(all_batches)}")
print("Age-Gender combinations:")
for i, batch in enumerate(all_batches, 1):
    print(f"{i:2d}. {batch['target_gender']} {batch['target_age']} ({len(batch['questions'])}Q, {len(batch['diagnosis'])}D)")

🔄 Processing: Male 0-2
  Filtering for Age: 0-2, Gender: Male
    ✓ Included Q1: What is your gender?...
    ✓ Included Q2: What is your age category?...
    ✓ Included Q3: Does the infant have a persistent cough?...
    ✓ Included Q4: Is the cough accompanied by fever?...
    ✓ Included Q5: Does the infant have difficulty breathing?...
    ✓ Included Q6: Is the cough wet or productive?...
    ✓ Included Q7: Does the infant have nasal congestion?...
    ✓ Included Q8: Is the infant's cough worse at night?...
    ✓ Included Q9: Is the infant wheezing while coughing?...
    ✓ Included Q10: Has the infant been exposed to smoke or pollutants...
    ✓ Included Q11: Does the infant have a history of allergies?...
    ✓ Included Q12: Does the infant vomit after coughing?...
    ✓ Included Q13: Is the infant feeding normally despite the cough?...
    ✓ Included Q14: Has the infant been recently vaccinated?...
    ✓ Included Diagnosis: Bronchiolitis
    ✓ Included Diagnosis: Croup
    ✓ Include

In [27]:
db_list = []

for i, batch in enumerate(all_batches, 1):
    age_group = batch['target_age']
    gender = batch['target_gender']
    
    print(f"⌛ [{i:2d}/{len(all_batches)}] Generating Decision Tree for {gender} {age_group}...")
    print(f"   📊 Input Data: {len(batch['questions'])} questions, {len(batch['diagnosis'])} diagnoses")
    
    dt_response = decision_tree_chain.invoke({"qa_json": batch})
    
    # Use model_dump() and extract ONLY the decision_tree
    db_data = dt_response.model_dump()
    db_list.extend(db_data['decision_tree'])  # Only the decision tree, not reasoning
    
    print(f"   ✅ Completed {gender} {age_group}\n")

⌛ [ 1/12] Generating Decision Tree for Male 0-2...
   📊 Input Data: 14 questions, 7 diagnoses
   ✅ Completed Male 0-2

⌛ [ 2/12] Generating Decision Tree for Female 0-2...
   📊 Input Data: 14 questions, 7 diagnoses
   ✅ Completed Female 0-2

⌛ [ 3/12] Generating Decision Tree for Male 3-12...
   📊 Input Data: 14 questions, 7 diagnoses
   ✅ Completed Male 3-12

⌛ [ 4/12] Generating Decision Tree for Female 3-12...
   📊 Input Data: 14 questions, 7 diagnoses
   ✅ Completed Female 3-12

⌛ [ 5/12] Generating Decision Tree for Male 13-18...
   📊 Input Data: 14 questions, 6 diagnoses
   ✅ Completed Male 13-18

⌛ [ 6/12] Generating Decision Tree for Female 13-18...
   📊 Input Data: 14 questions, 6 diagnoses
   ✅ Completed Female 13-18

⌛ [ 7/12] Generating Decision Tree for Male 19-40...
   📊 Input Data: 14 questions, 6 diagnoses
   ✅ Completed Male 19-40

⌛ [ 8/12] Generating Decision Tree for Female 19-40...
   📊 Input Data: 14 questions, 6 diagnoses
   ✅ Completed Female 19-40

⌛ [ 9/12] Ge

In [29]:
print(json.dumps(db_data['decision_tree'], indent=2))

[
  {
    "q_id": 1,
    "q_tag": "NEW",
    "question": "What is your gender?",
    "options": [
      {
        "next_q_id": 2,
        "opt_value": "Male",
        "option_id": 1
      },
      {
        "next_q_id": 2,
        "opt_value": "Female",
        "option_id": 2
      }
    ],
    "advice": null,
    "diagnosis": null,
    "lab_tests": null,
    "red_flags": null,
    "precautions": null,
    "otc_medication": null
  },
  {
    "q_id": 2,
    "q_tag": "NEW",
    "question": "What is your age category?",
    "options": [
      {
        "next_q_id": 72,
        "opt_value": "0-2 years (Infant/Toddler)",
        "option_id": 1
      },
      {
        "next_q_id": 72,
        "opt_value": "3-12 years (Child)",
        "option_id": 2
      },
      {
        "next_q_id": 72,
        "opt_value": "13-18 years (Adolescent)",
        "option_id": 3
      },
      {
        "next_q_id": 72,
        "opt_value": "19-40 years (Young Adult)",
        "option_id": 4
      },
      {

In [28]:
import pandas as pd
df=pd.DataFrame(data=db_list)
df.to_csv('drat_dt.csv',index=False)

In [15]:
import concurrent.futures
import time

def process_single_batch(batch):
    try:
        age_group = batch['questions'][2]['age_group']
        print(f"Generating Decision Tree for age group : {age_group}...")
        dt_response = decision_tree_chain.invoke({"qa_json": batch})
        db_data = dt_response.dict()
        print(f"Completed age group : {age_group}")
        return db_data['db_records']
    except Exception as e:
        print(f"Error processing batch: {e}")
        return []

print("Starting parallel processing...")
start_time = time.time()

db_list = []
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(process_single_batch, batch) for batch in all_batches]
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        db_list.extend(result)

end_time = time.time()
print(f"Completed in {end_time - start_time:.2f} seconds")
print(f"Total records: {len(db_list)}")

Starting parallel processing...
Generating Decision Tree for age group : 0-2...
Generating Decision Tree for age group : 0-2...
Generating Decision Tree for age group : 3-12...


C:\Users\Mukesh\AppData\Local\Temp\ipykernel_36028\3938038546.py:9: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  db_data = dt_response.dict()


Completed age group : 3-12
Generating Decision Tree for age group : 3-12...


C:\Users\Mukesh\AppData\Local\Temp\ipykernel_36028\3938038546.py:9: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  db_data = dt_response.dict()
C:\Users\Mukesh\AppData\Local\Temp\ipykernel_36028\3938038546.py:9: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  db_data = dt_response.dict()


Completed age group : 0-2
Generating Decision Tree for age group : 13-18...
Completed age group : 0-2
Generating Decision Tree for age group : 13-18...


...

Completed age group : 41-65


C:\Users\Mukesh\AppData\Local\Temp\ipykernel_36028\3938038546.py:9: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  db_data = dt_response.dict()


Completed age group : 66+
Completed age group : 66+


C:\Users\Mukesh\AppData\Local\Temp\ipykernel_36028\3938038546.py:9: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  db_data = dt_response.dict()


KeyboardInterrupt: 

In [ ]:
fp=open('cough_latest_complete.json','r')
qna_data=json.load(fp)

# Define age groups and genders for batching
age_groups = ['0-2', '3-12', '13-18', '19-40', '41-65', '66+']

all_batches=[]

def filter_data_for_batch(qa_data, target_age):
    """Filter questions and diagnoses for specific age group"""
    filtered_questions = []
    filtered_diagnoses = []
    
    # Filter questions
    for q in qa_data['questions']:
        q_age = q.get('age_group')
        
        # Include if: no age restriction, matches target age, or is demographic question
        if (q_age is None or q_age == target_age or q.get('q_tag') == 'DEMOGRAPHICS'):
            filtered_questions.append(q)
    
    # Filter diagnoses
    for d in qa_data['diagnosis']:
        d_age = d.get('age_group')
        
        if d_age == target_age:
            filtered_diagnoses.append(d)
    
    return {
        "questions": filtered_questions,
        "diagnosis": filtered_diagnoses
    }

for age_group in age_groups:
    print(f"Processing: {age_group}")
    
    # Filter data for this age group
    filtered_data = filter_data_for_batch(qna_data, age_group)
    all_batches.append(filtered_data)
    print(f"Filtered Data : {filtered_data}\n\n")

In [36]:
filtered_data

{'questions': [{'q_id': 1,
   'q_tag': 'DEMOGRAPHICS',
   'question': 'What is your gender?',
   'options': [{'option_id': 1, 'opt_value': 'Male'},
    {'option_id': 2, 'opt_value': 'Female'}],
   'age_group': None,
   'gender': 'Both',
   'clinical_rationale': 'Essential demographic information for accurate medical assessment'},
  {'q_id': 2,
   'q_tag': 'DEMOGRAPHICS',
   'question': 'What is your age category?',
   'options': [{'option_id': 1, 'opt_value': '0-2 years (Infant/Toddler)'},
    {'option_id': 2, 'opt_value': '3-12 years (Child)'},
    {'option_id': 3, 'opt_value': '13-18 years (Adolescent)'},
    {'option_id': 4, 'opt_value': '19-40 years (Young Adult)'},
    {'option_id': 5, 'opt_value': '41-65 years (Middle-aged Adult)'},
    {'option_id': 6, 'opt_value': '66+ years (Senior)'}],
   'age_group': None,
   'gender': 'Both',
   'clinical_rationale': 'Age-specific medical assessment and treatment considerations'},
  {'q_id': 72,
   'q_tag': 'SYMPTOMS',
   'question': 'Does 

# NEW APPROACH

In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List, Optional
import json

In [2]:
class Option(BaseModel):
    option_id: int = Field(description="Unique identifier for this option")
    opt_value: str = Field(description="The option text displayed to user")
    next_q_id: Optional[int] = Field(description="Next question ID - can be question or diagnosis")

class DiagnosisInfo(BaseModel):
    diagnosis_title: str = Field(description="Name of the diagnosis")
    description: str = Field(description="Detailed description of the condition")

class DecisionTreeNode(BaseModel):
    id: int = Field(description="Unique node identifier for mobile app navigation")
    q_id: int = Field(description="Question/diagnosis identifier")
    q_tag: str = Field(description="Node type: 'GENERAL', 'SYMPTOMS', or 'DIAGNOSIS'")
    question: str = Field(description="Question text or diagnosis statement")
    options: List[Option] = Field(default=[], description="Available options (empty for diagnosis)")
    
    # Diagnosis-specific fields (only for diagnosis nodes)
    diagnosis: Optional[DiagnosisInfo] = Field(default=None)
    advice: Optional[List[str]] = Field(default=None)
    lab_tests: Optional[List[str]] = Field(default=None)
    red_flags: Optional[List[str]] = Field(default=None)
    precautions: Optional[List[str]] = Field(default=None)
    otc_medication: Optional[List[str]] = Field(default=None)

class Reasoning(BaseModel):
    node_id: int = Field(description="Current node ID being reasoned about")
    q_id: int = Field(description="Question ID from original medical data")
    option_id: int = Field(description="Option ID that triggers this reasoning path")
    next_node_id: int = Field(description="Next node ID this option leads to")
    clinical_rule: str = Field(description="Clinical decision rule explaining why this option leads to next step")
    medical_validation: str = Field(description="Medical validation confirming this pathway follows clinical guidelines")
    risk_assessment: str = Field(description="Risk assessment for this pathway (low/medium/high priority)")
    conclusion: str = Field(description="Final clinical reasoning with evidence-based justification")

class SmartDecisionTreeWithReasoning(BaseModel):
    decision_tree: List[DecisionTreeNode] = Field(description="Complete smart decision tree")
    reasoning_steps: List[Reasoning] = Field(description="Clinical reasoning validation for each decision pathway")
    tree_metadata: dict = Field(description="Metadata about tree structure and validation")